# Objetivo

Desarrollo Parcial practico - Aprendizaje automatico III - ICESI

# Instrucciones
El examen se compone de dos partes:
- La primera corresponde a una parte de selección múltiple con 19 preguntas que se responderán en el salón de clase en 45 minutos. Esa primera parte tiene una calificación de 1 a 5.
- La segunda parte corresponde a la parte práctica del examen y tiene también una calificación de 1 a 5.

- La nota de este examen parcial corresponderá al promedio ponderado de las dos notas, donde la parte práctica tiene un peso de 40 % y la parte de selección múltiple de 60 %.
- Usted tiene hasta el 30 de junio a las 23:59  para enviar los archivos por correo, estos archivos deben tener su nombre.
- Sólo se calificaran exámenes en formato pdf. Cualquier otro formato no será tenido en cuenta.
- El examen debe estar acompañado de un notebook (ipynb)  que incluya todo los códigos de python que se emplean para obtener sus resultados.
- El nombre del archivo debe tener su nombre. No se recibirán archivos en otro formato.
- Esta parte del examen es para realizar en casa y debe reflejar el trabajo individual.

## El Problema

En días recientes trabajamos con una compañía de comestibles que estaba interesada en predecir el comportamiento de las ventas (en unidades) de sus dos productos estrella. Fuimos contratados para generar un modelo que permita pronosticar las ventas del siguiente mes de cada uno de esos dos productos. La base de datos disponible en el archivo Examen.csv tiene la información de cada uno de los productos desde junio de 2014.

Su misión es encontrar el mejor modelo para pronosticar cada una de las series. Usted debe entregar un informe escrito de no más de cuatro páginas que presente los resultados al cliente y cuente el proceso para llegar a los pronósticos. Vea las instrucciones para asegurar que entrega los archivos requeridos

# Lectura de paquetes

In [ ]:
import logging

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt  # gráficos
import pylab as py
from scipy import stats
from datetime import datetime
from dateutil.relativedelta import relativedelta

from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

import optuna

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch
from statsmodels.stats.stattools import jarque_bera
from scipy.stats import shapiro
import statsmodels.api as sm

import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
optuna.logging.set_verbosity(logging.WARNING)  # Solo muestra warnings o errores


from src.ml_utils import *
from src.plot_utils import *

# colocar estilo fondo blanco
plt.style.use("default")

# Lectura de data

In [ ]:
df = pd.read_csv("Examen.csv").iloc[:,1:]

# Crear un rango de fechas mensuales desde junio 2014 por 127 meses
fechas = pd.date_range(start="2014-06-01", periods=127, freq="MS")
df.index = fechas
df.index.name = "fecha"


In [ ]:
df.info()

# Analisis Exploratorio (EDA)

In [ ]:
plt.figure(figsize=(13,5))
plt.subplot(1,2,1)
df["producto1"].plot()
plt.title("Producto 1")
plt.ylabel("Ventas (unidades)")
plt.xticks(rotation=45)
plt.grid()

plt.subplot(1,2,2)
df["producto2"].plot()
plt.title("Producto 2")
plt.ylabel("Ventas (unidades)")
plt.xticks(rotation=45)
plt.grid()

## Descomposición STL

In [ ]:
# Descomposición STL de ambas series

stl_producto1 = STL(df["producto1"], seasonal=13)
stl_producto2 = STL(df["producto2"], seasonal=13)
result_producto1 = stl_producto1.fit()
result_producto2 = stl_producto2.fit()

df["producto1_trend"] = result_producto1.trend
df["producto1_seasonal"] = result_producto1.seasonal
df["producto2_trend"] = result_producto2.trend
df["producto2_seasonal"] = result_producto2.seasonal

# graficar usando plotly
graficar_serie_con_descomposicion(df, "producto #1", "producto1")
graficar_serie_con_descomposicion(df, "producto #2", "producto2")

- No se nota con claridad efectos estacionales.  
- Podemos notar una clara y marcada tendencia en ambas series de productos.

# Planeación

Para dar solución a lo solicitado, procederé con los siguientes pasos y aspectos a tener en cuenta:
- Se empleará una estrategia de selección de modelos jerarquica, dividida en 3 pasos o filtros. El primer paso será "evaluación", luego "validación" y por ultimo "test final".
- Para todo el proceso de ajuste y selección de modelos, usaré solo los ultimos 6 años de los datos, dado que, las series presentan cambios a lo largo del tiempo.
- Para el paso de evaluación de modelos se usarán los primeros 6 meses del ultimo año de la data. Para la validación usaré del mes #7 al #10 del ultimo año y para el test final usaré los ultimos 2 meses.
-  En el paso de evaluación se filtrarán los 20 mejores modelos obtenidos, que si lo requieren, tambien cumplan los supuestos. En el paso de validación se filtrará a los mejores 3 modelos por serie y para el test final, se probarán alternativas de unión de predicciones junto a los mejores 3 modelos, para elegir la estrategia o modelo ganador.
- Para el calculo de metricas, usaré una ventana movil con calculos mes a mes, dado que, en la practica asi desea hacer las predicciones el cliente.
- Se usarán estrategias de optimización para identificar las mejores combinaciones de hiperparametros para los modelos, tales como, gridsearch y optuna.
- Las familias de modelos y estrategias de optimización que se emplearán son: Modelos de suavización exponencial (Optuna), modelos ARIMA (gridsearch y validación de supuestos), Promedio movil (de 2 a 12), Regresión lineal multiple con estacionalidad de 12 meses (Grado 1 a grado 5, y validación de supuestos), Prophet (Optuna).
- Para evitar overfitting se utilizarán estrategias de validación cruzada, y de minimización de diferencias entre metricas en el conjunto de entrenamiento y el conjunto sobre el que se esté prediciendo.
- Por ultimo, Las metricas que se usarán para cuantificar el desempeño de los modelos serán MAPE, RMSE y R2.

# Preparación de la data (FE)

In [ ]:
# Data para paso de evaluación - desde 6 años atras hasta 6 meses antes del final
df_evaluacion = df.iloc[-1*(6*12):-6, 0:2].copy()
df_producto1_evaluacion = df_evaluacion["producto1"].copy()
df_producto2_evaluacion = df_evaluacion["producto2"].copy()

# Data para paso de validación - desde 6 meses antes del final hasta 2 meses antes del final
df_validacion = df.iloc[-1*(6*12):-2, 0:2].copy()
df_producto1_validacion = df_validacion["producto1"].copy()
df_producto2_validacion = df_validacion["producto2"].copy()

# Data para paso de test - ultimos 2 meses
df_test = df.iloc[-2:, 0:2].copy()
df_producto1_test = df_test["producto1"].copy()
df_producto2_test = df_test["producto2"].copy()

In [ ]:
print("Tamaño de data de evaluacion, validación y test:")
df_producto1_evaluacion.shape,df_producto2_evaluacion.shape, df_producto1_validacion.shape, df_producto2_validacion.shape, df_producto1_test.shape, df_producto2_test.shape

# 1. Evaluación inicial de modelos

En esta primera etapa, realizaré un primer filtro de los mejores 20 modelos validos y utiles para los dos productos, realizando busqueda u optimización de los mejores parametros de los modelos.  
Además, realizaré validación de supuestos sobre el error, para modelos ARIMA y RLM implementados con el objetivo de filtrar aquellos validos.

## ETS models

### producto 1

In [ ]:
# Inicializamos parametros para evaluación por ventana recursiva (expansiva)
test_init = "2024-01-01"
test_finish = "2024-06-01"
window_type = "rolling"
train_size = (12*5) # 5 años
test_size = 1
metric = "rmse"

# Optimizamos el modelo ETS usando Optuna
study, best_params, rmse_movil = optimizar_modelo_ets(
    df_producto1_evaluacion,
    test_init,
    test_finish,
    window_type,
    train_size,
    test_size,
    metric,
    opt_trial=500,
)

In [ ]:
ets_prod1_results = study.trials_dataframe()
ets_prod1_results.sort_values(by="value", ascending=True, inplace=True)
ets_prod1_results = ets_prod1_results[
    [
        "value",
        "params_trend",
        "params_seasonal",
        "params_seasonal_periods",
        "params_damped_trend",
        "params_use_box_cox",
        "params_smoothing_level",
        "params_smoothing_trend",
        "params_smoothing_seasonal",
        "params_damping_trend",
    ]
]
ets_prod1_results.columns = (
    "rmse",
    "trend",
    "seasonal",
    "seasonal_periods",
    "damped_trend",
    "boxcox",
    "alpha",
    "beta",
    "gamma",
    "damping_trend",
)
ets_prod1_results.head(5)

### producto 2

In [ ]:
# Inicializamos parametros para evaluación por ventana recursiva (expansiva)
test_init = "2024-01-01"
test_finish = "2024-06-01"
window_type = "rolling"
train_size = (12*5) # 5 años
test_size = 1
metric = "rmse"

# Optimizamos el modelo ETS usando Optuna
study, best_params, rmse_movil = optimizar_modelo_ets(
    df_producto2_evaluacion,
    test_init,
    test_finish,
    window_type,
    train_size,
    test_size,
    metric,
    opt_trial=500,
)


In [ ]:
ets_prod2_results = study.trials_dataframe()
ets_prod2_results.sort_values(by="value", ascending=True, inplace=True)
ets_prod2_results = ets_prod2_results[
    [
        "value",
        "params_trend",
        "params_seasonal",
        "params_seasonal_periods",
        "params_damped_trend",
        "params_use_box_cox",
        "params_smoothing_level",
        "params_smoothing_trend",
        "params_smoothing_seasonal",
        "params_damping_trend",
    ]
]
ets_prod2_results.columns = (
    "rmse",
    "trend",
    "seasonal",
    "seasonal_periods",
    "damped_trend",
    "boxcox",
    "alpha",
    "beta",
    "gamma",
    "damping_trend",
)
ets_prod2_results.head(5)

## ARIMA models

### Producto 1

In [ ]:
# Funcion para ajustar modelos ARIMA con evaluación por ventana
arima_prod1_results = ajustar_modelos_arima(
    df=df_producto1_evaluacion,
    test_init="2024-01-01",
    test_finish="2024-06-01",
    p_range=range(0,8),
    d_range=range(0,3),
    q_range=range(0,8),
)

# Función para validar supuestos de ARIMA
arima_prod1_results = validar_supuestos_df(df_producto1_evaluacion, arima_prod1_results, "ARIMA", alpha=0.05)
arima_prod1_results = arima_prod1_results[arima_prod1_results["validar_supuestos"] == True].drop(columns=["validar_supuestos"]).sort_values(by="rmse", ascending=True)
arima_prod1_results.head(5)

### Producto 2

In [ ]:
arima_prod2_results = ajustar_modelos_arima(
    df=df_producto2_evaluacion,
    test_init="2024-01-01",
    test_finish="2024-06-01",
    p_range=range(0,8),
    d_range=range(0,3),
    q_range=range(0,8),
)

arima_prod2_results = validar_supuestos_df(df_producto2_evaluacion, arima_prod2_results, "ARIMA", alpha=0.05)
arima_prod2_results = arima_prod2_results[arima_prod2_results["validar_supuestos"] == True].drop(columns=["validar_supuestos"]).sort_values(by="rmse", ascending=True)
arima_prod2_results.head(5)

## Mean Average models

### Producto 1

In [ ]:
ma_prod1_results = pd.DataFrame(columns=["size", "rmse"])

# Evaluamos el modelo MA para diferentes tamaños de ventana 
for size in range(2,12):
    fit_params = {"window_size":size, "horizon":1}

    metric = evalua_modelo_ST_por_ventana(
        index_name="fecha",
        model_type="MA",
        init_params={},
        fit_params=fit_params,
        data=df_producto1_evaluacion,
        test_init="2024-01-01",
        test_finish="2024-06-01",
        window_type="rolling",
        train_size=12 * 5,  # 5 años
        test_size=1,
        metric="rmse",
    )

    result = pd.DataFrame(
        {
            "size": size,
            "rmse": metric,
        }, index=[0]
    )

    ma_prod1_results = pd.concat([ma_prod1_results, result], ignore_index=True)

ma_prod1_results.sort_values(by="rmse", ascending=True, inplace=True)
ma_prod1_results

### Producto 2

In [ ]:
ma_prod2_results = pd.DataFrame(columns=["size", "rmse"])

# Evaluamos el modelo MA para diferentes tamaños de ventana 
for size in range(2,12):
    fit_params = {"window_size":size, "horizon":1}

    metric = evalua_modelo_ST_por_ventana(
        index_name="fecha",
        model_type="MA",
        init_params={},
        fit_params=fit_params,
        data=df_producto2_evaluacion,
        test_init="2024-01-01",
        test_finish="2024-06-01",
        window_type="rolling",
        train_size=12 * 5,  # 5 años
        test_size=1,
        metric="rmse",
    )

    result = pd.DataFrame(
        {
            "size": size,
            "rmse": metric,
        }, index=[0]
    )

    ma_prod2_results = pd.concat([ma_prod2_results, result], ignore_index=True)

ma_prod2_results.sort_values(by="rmse", ascending=True, inplace=True)
ma_prod2_results

## RLM models

## Prophet

# Next

In [ ]:
# # creamos el diccionario de parametros para inicializar el modelo ETS
#         init_params = {
#             "trend": trend,
#             "damped_trend": damped_trend,
#             "seasonal": seasonal,
#             "seasonal_periods": seasonal_periods,
#             "use_boxcox": use_box_cox,
#         }

#         # Creamos el diccionario de parametros para ajustar el modelo ETS
#         fit_params = {
#             "smoothing_level": slevel,
#             "smoothing_trend": strend,
#             "smoothing_seasonal": sseasonal,
#             "damping_trend": damping_value,
#         }
